## Data cleaning

In [1]:
import csv
import pandas as pd

In [2]:
df = pd.read_csv("flats_info.csv")

In [3]:
df.shape

(2416, 7)

In [4]:
# Add prefix for original columns
df = df.add_prefix('col_')

In [5]:
df.head()

,col_room_number,col_region,col_area,col_price,col_price_sq_m,col_floor,col_layout
0,1-комн.,"Кировский р-н, Кутузова, 1",38.60 м 2,3 474 000 руб.,90 000 руб./м2,эт. из 20,"Кировский р-н, Кутузова, 1\nПланировка: новая\..."
1,1-комн.,"Советский р-н, Шумяцкого, 7",29.70 м 2,3 340 000 руб.,112 458 руб./м2,1 эт. из 9,"Советский р-н, Шумяцкого, 7\nПланировка: новая..."
2,3-комн.,"Советский р-н, 60 лет Образования СССР, 20",64.10 м 2,6 500 000 руб.,101 404 руб./м2,2 эт. из 9,"Советский р-н, 60 лет Образования СССР, 20\nПл..."
3,3-комн.,"Свердловский р-н, Семафорная, 241",58.40 м 2,4 990 000 руб.,85 445 руб./м2,1 эт. из 5,"Свердловский р-н, Семафорная, 241\nПланировка:..."
4,3-комн.,"Свердловский р-н, Южная набережная, 10",92.30 м 2,12 400 000 руб.,134 345 руб./м2,5 эт. из 6,"Свердловский р-н, Южная набережная, 10\nПланир..."


In [6]:
# Check for duplicate variables
df.duplicated().sum()

172

In [7]:
# Remove duplicates
df = df.drop_duplicates()

### flat type

In [8]:
# Delete data with 'NI' - not info values
df = df[df.col_room_number != 'NI']

In [9]:
# Rename values / Translation
df['flat_type']=df['col_room_number'].replace({'1-комн.': '1_room_ap', '2-комн.': '2_room_ap', '3-комн.': '3_room_ap',
                 '4-комн.': '4_room_ap', '5-комн.': '5_room_ap', 'Гостин.': 'gostinka_ap',
              'Комнат.': 'room', 'Секцио.': 'sectional_ap'})

### city area

In [10]:
city_area = df['col_region'].apply(lambda x: x.split(',')[0])

In [11]:
df['city_area'] = city_area.apply(lambda x: x.replace(' р-н',''))

In [12]:
# Delete values that are not related to the area of the city of Krasnoyarsk
df = df[df.city_area != 'Краснодарский край ']
df = df[df.city_area != 'Дивногорский ']
df = df[df.city_area != 'Сосновоборский ']
df = df[df.city_area != 'Рыбинский ']
df = df[df.city_area != 'Березовский ']
df = df[df.city_area != 'Сухобузимский ']
df = df[df.city_area != 'Железногорский ']
df = df[df.city_area != 'Емельяновский ']

In [13]:
# Rename values / Translation
df['city_area']=df['city_area'].replace({'Советский': 'Sovietskiy', 'Октябрьский': 'Oktyabrskiy', 
                                         'Свердловский': 'Sverdlovsky', 'Ленинский': 'Leninsky', 
                                         'Железнодорожный': 'Zheleznodorozhniy', 'Кировский': 'Kirovsky',
                                         'Центральный': 'Central', 'Емельяновский ': 'Emelyanovsky'})

###  premises area

In [14]:
df['premises_area'] = df['col_area'].apply(lambda x: x.split('м')[0]).astype(float)

### floor and total floor

We divide the raw col_floor column into two: one of which will be floor, the second total floor in the building.

In [15]:
df['floor'] = df['col_floor'].apply(lambda x: x.split('эт. из ')[0])

In [16]:
# Delete data with empty value
df = df[df.floor != '']
df['floor'] = df['floor'].astype(int)

In [17]:
# Total floors in the building
df['floor_tot'] = df['col_floor'].apply(lambda x: x.split('эт. из ')[1]).astype(int)

### layout and status of the construction

We divide the raw col_layout column into two: one of which will mean the layout, the second the status of the building.

In [18]:
df['layout'] = df['col_layout'].apply(lambda x: x.split('\n')[1])

In [19]:
# Delete one line in which information about the type of layout is missing
df = df[df['layout'] != 'Готовое']

In [20]:
df['layout'] = df['layout'].apply(lambda x: x.split('Планировка: ')[1])

In [21]:
# Rename values / Translation
df['layout']=df['layout'].replace({'новая': 'new', 'студия': 'studio', 
                                         'индивидуальная': 'individual', 'улучшенная': 'improved', 
                                         'хрущевка': 'khrushchevka', 'сталинка': 'stalinka',
                                         'ленинградка': 'leningradka', 'гостинка': 'gostinka',
                                  'комната в общежитии': 'dorm_room', 'секционка': 'section',
                                  '2х-уровневая': '2-level', 'комната в квартире': 'room_in_flat',
                                  'апартаменты': 'studio'})

In [22]:
df['status'] = df['col_layout'].apply(lambda x: x.split('\n')[2])

In [23]:
# Rename values / Translation
df['status']=df['status'].replace({'Готовое': 'finished', 'Строится': 'under_construction'})

### price

In [24]:
df['price'] = df['col_price'].apply(lambda x: x.split('р')[0])

In [25]:
# Remove spaces
df['price'] = df['price'].apply(lambda x: x.replace(' ','')).astype(int)

In [26]:
# Convert to euro
df['price'] = df['price']/86

In [27]:
df.head()

,col_room_number,col_region,col_area,col_price,col_price_sq_m,col_floor,col_layout,flat_type,city_area,premises_area,floor,floor_tot,layout,status,price
1,1-комн.,"Советский р-н, Шумяцкого, 7",29.70 м 2,3 340 000 руб.,112 458 руб./м2,1 эт. из 9,"Советский р-н, Шумяцкого, 7\nПланировка: новая...",1_room_ap,Sovietskiy,29.7,1,9,new,finished,38837.209302
2,3-комн.,"Советский р-н, 60 лет Образования СССР, 20",64.10 м 2,6 500 000 руб.,101 404 руб./м2,2 эт. из 9,"Советский р-н, 60 лет Образования СССР, 20\nПл...",3_room_ap,Sovietskiy,64.1,2,9,new,finished,75581.395349
3,3-комн.,"Свердловский р-н, Семафорная, 241",58.40 м 2,4 990 000 руб.,85 445 руб./м2,1 эт. из 5,"Свердловский р-н, Семафорная, 241\nПланировка:...",3_room_ap,Sverdlovsky,58.4,1,5,khrushchevka,finished,58023.255814
4,3-комн.,"Свердловский р-н, Южная набережная, 10",92.30 м 2,12 400 000 руб.,134 345 руб./м2,5 эт. из 6,"Свердловский р-н, Южная набережная, 10\nПланир...",3_room_ap,Sverdlovsky,92.3,5,6,new,finished,144186.046512
5,2-комн.,"Октябрьский р-н, Норильская, 8г",48.00 м 2,4 300 000 руб.,89 583 руб./м2,10 эт. из 17,"Октябрьский р-н, Норильская, 8г\nПланировка: н...",2_room_ap,Oktyabrskiy,48.0,10,17,new,finished,50000.000000


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2207 entries, 1 to 2415
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   col_room_number  2207 non-null   object 
 1   col_region       2207 non-null   object 
 2   col_area         2207 non-null   object 
 3   col_price        2207 non-null   object 
 4   col_price_sq_m   2207 non-null   object 
 5   col_floor        2207 non-null   object 
 6   col_layout       2207 non-null   object 
 7   flat_type        2207 non-null   object 
 8   city_area        2207 non-null   object 
 9   premises_area    2207 non-null   float64
 10  floor            2207 non-null   int64  
 11  floor_tot        2207 non-null   int64  
 12  layout           2207 non-null   object 
 13  status           2207 non-null   object 
 14  price            2207 non-null   float64
dtypes: float64(2), int64(2), object(11)
memory usage: 275.9+ KB


In [29]:
df_save = df[['flat_type', 'city_area', 'premises_area', 'floor', 'floor_tot', 'layout', 'status', 'price']]

In [30]:
# save results to csv
df_save.to_csv("flats_info_cleaned.csv", index=False)